# The Notebook Calculates A Simulation Model
<font color="red">MUST RUN DOWNLOAD CMS Files notebook first</font>

This notebook will use the Medicare Part B files to calculate a simulation of what the Medicare Physician Fee Schedule proposal. Once this notebook is finished, it will save the model using the medicare_payment_model_saved variable as the file name.

The calculation take a fair amount of time. It may take the notebook <font color="red">over 10 minutes</font> to complete.

In [1]:
medicare_saved = "medicare_saved.pkl"
medicare_agg_saved = "medicare_agg_saved.pkl"

medicare_payment_model_saved = "medicare_payment_model.pkl"

In [2]:
import numpy as np
import pandas as pd

These are the two formulas that calculate the model. The `calculateRates` function contains the calculations that depend on the CPT code. The `calculateProposedRates` are the calculations that are based on the per NPI. 

Currently, the model only calculates for Primary Care (only Family Physicians currently). There are some assumptions made:
 * Time saved are only calculated for Level 3-5 visits & 100% is converted to saved dollars
 * To estimate the number of prolonged visits, use 70% of 99205 and 50% of 99215
 * Family Physician average hourly income is $93/hr

In [3]:
def calculateRates(row):
    pc_rate = 5 #Primary Care G-Code Rate
    spec_rate = 9 #Specialty G-Code Rate
    
    # [Current Payment Rate, Proposed Payment Rate, Fraction of encounters that primary care Gcode applies
    #        , Fraction of encounters where time is saved, Fraction of encounters that might be > 16 min ]
    rowRates = {
        '99201':[45,44,0,0,0],'99202':[76,135,0,0,0],
        '99203':[110,13,0,1,0],'99204':[167,135,0,1,0],'99205':[211,135,0,1,0.7],
        '99211':[22,24,pc_rate,0,0],'99212':[45,93,pc_rate,0,0],
        '99213':[74,93,pc_rate,1,0],'99214':[109,93,pc_rate,1,0],'99215':[148,93,pc_rate,1,0.5]
    }.get(row['HCPCS_CODE'])

    row['CURRENT_RATE'] = row['LINE_SRVC_CNT'] * rowRates[0] # Current Payment
    row['PROPOSED_RATE'] = row['LINE_SRVC_CNT'] * rowRates[1] # Proposed Payment
    
    if row['PROVIDER_TYPE'] == 'Family Practice':
        row['PRIMARYCARE_GCODE'] = row['LINE_SRVC_CNT'] * rowRates[2] # Primary Care G-Code Payment
        row['PRIMARYCARE_GCODE_PLUS5'] = row['LINE_SRVC_CNT'] * (rowRates[2] + 5) # Primary Care G-Code Payment
        row['PRIMARYCARE_GCODE_PLUS8'] = row['LINE_SRVC_CNT'] * (rowRates[2] + 8) # Primary Care G-Code Payment
        row['PRIMARYCARE_GCODE_PLUS10'] = row['LINE_SRVC_CNT'] * (rowRates[2] + 10) # Primary Care G-Code Payment
    
    row['MIN_SAVED'] = 1.6 * rowRates[3] * row['LINE_SRVC_CNT']
    
    #Calculate Prolonged Visits
    row['PROLONGED_VISIT_GCODE'] = row['LINE_SRVC_CNT'] * rowRates[4] * 67
    return row

def calculateProposedRates(row):
    # Add together the proposed base rate and the primary care complexity g-code rate
    if row['PROVIDER_TYPE'] == 'Family Practice':
        row['PROPOSED_RATE_WITH_GCODE'] = row['PROPOSED_RATE'] + row['PRIMARYCARE_GCODE']
        row['PROPOSED_RATE_WITH_GCODE_PLUS5'] = row['PROPOSED_RATE'] + row['PRIMARYCARE_GCODE_PLUS5']
        row['PROPOSED_RATE_WITH_GCODE_PLUS8'] = row['PROPOSED_RATE'] + row['PRIMARYCARE_GCODE_PLUS8']
        row['PROPOSED_RATE_WITH_GCODE_PLUS10'] = row['PROPOSED_RATE'] + row['PRIMARYCARE_GCODE_PLUS10']

    
    # Calculate the difference between the proposed new rate and old current rate WITHOUT primary care g-code
    row['DIFFERENCE'] = row['PROPOSED_RATE'] - row['CURRENT_RATE']
    # Calculate the difference between the proposed new rate and old current rate WITH primary care g-code
    if row['PROVIDER_TYPE'] == 'Family Practice':
        row['DIFFERENCE_WITH_GCODE'] = row['PROPOSED_RATE_WITH_GCODE'] - row['CURRENT_RATE']
        row['DIFFERENCE_WITH_GCODE_PLUS5'] = row['PROPOSED_RATE_WITH_GCODE_PLUS5'] - row['CURRENT_RATE']
        row['DIFFERENCE_WITH_GCODE_PLUS8'] = row['PROPOSED_RATE_WITH_GCODE_PLUS8'] - row['CURRENT_RATE']
        row['DIFFERENCE_WITH_GCODE_PLUS10'] = row['PROPOSED_RATE_WITH_GCODE_PLUS10'] - row['CURRENT_RATE']

    # Create a new column with the payment difference normalize to number of unique beneficaries per FP
    row['DIFFERENCE_WITH_GCODE_PER_BENE'] = row['DIFFERENCE_WITH_GCODE'] / row['TOTAL_UNIQUE_BENES']
    if row['PROVIDER_TYPE'] == 'Family Practice':
        row['PROPOSED_RATE_WITH_GCODE_PER_BENE'] = row['PROPOSED_RATE_WITH_GCODE'] / row['TOTAL_UNIQUE_BENES']
        row['DIFFERENCE_WITH_GCODE_PLUS5_PER_BENE'] = row['DIFFERENCE_WITH_GCODE_PLUS5'] / row['TOTAL_UNIQUE_BENES']
        row['DIFFERENCE_WITH_GCODE_PLUS8_PER_BENE'] = row['DIFFERENCE_WITH_GCODE_PLUS8'] / row['TOTAL_UNIQUE_BENES']
        row['DIFFERENCE_WITH_GCODE_PLUS10_PER_BENE'] = row['DIFFERENCE_WITH_GCODE_PLUS10'] / row['TOTAL_UNIQUE_BENES']
        #Calculate $ equivalent to time saving. Assume $93/hr for FP hourly rate
        row['DOLLARS_SAVED'] = row['MIN_SAVED']* 1.55

    return row

In [4]:
df_medicare = pd.read_pickle(medicare_saved)
df_medicare.head()

,NPI,PLACE_OF_SERVICE,HCPCS_CODE,LINE_SRVC_CNT,PROVIDER_TYPE
0,1003000126,F,99217,57.0,Internal Medicine
1,1003000126,F,99219,38.0,Internal Medicine
2,1003000126,F,99220,23.0,Internal Medicine
3,1003000126,F,99221,20.0,Internal Medicine
4,1003000126,F,99222,96.0,Internal Medicine


Currently, the notebook focuses only on Family Practice and the E&M codes in the list

In [5]:
df_med_filtered = df_medicare.query('PROVIDER_TYPE == "Family Practice"')
df_med_filtered = df_med_filtered[df_med_filtered.HCPCS_CODE.isin(['99201', '99202','99203','99204','99205','99211', '99212','99213','99214','99215'])]
df_med_filtered.head()

,NPI,PLACE_OF_SERVICE,HCPCS_CODE,LINE_SRVC_CNT,PROVIDER_TYPE
66,1003000522,O,99204,16.0,Family Practice
67,1003000522,O,99211,41.0,Family Practice
68,1003000522,O,99212,12.0,Family Practice
69,1003000522,O,99213,330.0,Family Practice
70,1003000522,O,99214,612.0,Family Practice


This cell will take some time to complete.

In [6]:
df_med_filtered = df_med_filtered.apply(calculateRates, axis=1)

In [7]:
df_medicare_summed = df_med_filtered.groupby('NPI').agg({ 'LINE_SRVC_CNT':sum, 'CURRENT_RATE':sum, 'PROPOSED_RATE':sum, 'PRIMARYCARE_GCODE':sum,
                              'PRIMARYCARE_GCODE_PLUS5':sum, 'PRIMARYCARE_GCODE_PLUS8':sum, 'PRIMARYCARE_GCODE_PLUS10':sum, 'PROLONGED_VISIT_GCODE':sum, 'MIN_SAVED':sum})

In [8]:
df_medicare_agg = pd.read_pickle(medicare_agg_saved)
df_merged = pd.merge(df_medicare_summed, df_medicare_agg, on='NPI', how='left')
df_merged.head()

,NPI,LINE_SRVC_CNT,CURRENT_RATE,PROPOSED_RATE,PRIMARYCARE_GCODE,PRIMARYCARE_GCODE_PLUS5,PRIMARYCARE_GCODE_PLUS8,PRIMARYCARE_GCODE_PLUS10,PROLONGED_VISIT_GCODE,MIN_SAVED,NPPES_ENTITY_CODE,BENEFICIARY_AVERAGE_RISK_SCORE,NPPES_PROVIDER_STATE,TOTAL_UNIQUE_BENES,PROVIDER_TYPE
0,1003000522,1034.0,98646.0,94005.0,5090.0,10260.0,13362.0,15430.0,770.5,1569.6,I,1.2619,FL,417,Family Practice
1,1003000902,684.0,56788.0,60972.0,3255.0,6675.0,8727.0,10095.0,0.0,1072.0,I,1.2416,KY,258,Family Practice
2,1003001256,103.0,9057.0,9579.0,515.0,1030.0,1339.0,1545.0,0.0,164.8,I,2.2628,CO,147,Family Practice
3,1003001884,250.0,23810.0,23250.0,1250.0,2500.0,3250.0,3750.0,502.5,400.0,I,1.1660,MI,145,Family Practice
4,1003002817,572.0,52163.0,53196.0,2860.0,5720.0,7436.0,8580.0,0.0,915.2,I,1.7376,FL,207,Family Practice


This cell will take some time to complete.

In [9]:
df_merged = df_merged.apply(calculateProposedRates, axis=1)

In [10]:
df_merged.head()

,NPI,LINE_SRVC_CNT,CURRENT_RATE,PROPOSED_RATE,PRIMARYCARE_GCODE,PRIMARYCARE_GCODE_PLUS5,PRIMARYCARE_GCODE_PLUS8,PRIMARYCARE_GCODE_PLUS10,PROLONGED_VISIT_GCODE,MIN_SAVED,...,DIFFERENCE_WITH_GCODE,DIFFERENCE_WITH_GCODE_PLUS5,DIFFERENCE_WITH_GCODE_PLUS8,DIFFERENCE_WITH_GCODE_PLUS10,DIFFERENCE_WITH_GCODE_PER_BENE,PROPOSED_RATE_WITH_GCODE_PER_BENE,DIFFERENCE_WITH_GCODE_PLUS5_PER_BENE,DIFFERENCE_WITH_GCODE_PLUS8_PER_BENE,DIFFERENCE_WITH_GCODE_PLUS10_PER_BENE,DOLLARS_SAVED
0,1003000522,1034.0,98646.0,94005.0,5090.0,10260.0,13362.0,15430.0,770.5,1569.6,...,449.0,5619.0,8721.0,10789.0,1.076739,237.637890,13.474820,20.913669,25.872902,2432.88
1,1003000902,684.0,56788.0,60972.0,3255.0,6675.0,8727.0,10095.0,0.0,1072.0,...,7439.0,10859.0,12911.0,14279.0,28.833333,248.941860,42.089147,50.042636,55.344961,1661.60
2,1003001256,103.0,9057.0,9579.0,515.0,1030.0,1339.0,1545.0,0.0,164.8,...,1037.0,1552.0,1861.0,2067.0,7.054422,68.666667,10.557823,12.659864,14.061224,255.44
3,1003001884,250.0,23810.0,23250.0,1250.0,2500.0,3250.0,3750.0,502.5,400.0,...,690.0,1940.0,2690.0,3190.0,4.758621,168.965517,13.379310,18.551724,22.000000,620.00
4,1003002817,572.0,52163.0,53196.0,2860.0,5720.0,7436.0,8580.0,0.0,915.2,...,3893.0,6753.0,8469.0,9613.0,18.806763,270.801932,32.623188,40.913043,46.439614,1418.56


Calculate some columns for model analysis

In [12]:
df_merged['DIFFERENCE_WITH_GCODE_PLUS_SAVED'] = df_merged['DIFFERENCE_WITH_GCODE']+df_merged['DOLLARS_SAVED']
df_merged['DIFFERENCE_WITH_GCODE_PLUS_PROLONGED'] = df_merged['DIFFERENCE_WITH_GCODE']+df_merged['PROLONGED_VISIT_GCODE']
df_merged['PROPOSED_RATE_FULL'] = df_merged['PROPOSED_RATE'] + df_merged['PRIMARYCARE_GCODE'] + df_merged['PROLONGED_VISIT_GCODE'] + df_merged['DOLLARS_SAVED']
df_merged['BENEFICIARY_AVERAGE_RISK_SCORE_BINNED'] = pd.cut(df_merged['BENEFICIARY_AVERAGE_RISK_SCORE'], 3, labels=['low', 'medium', 'high'])

In [13]:
df_merged.to_pickle(medicare_payment_model_saved)